In [ ]:
import requests
import json
import time
import itertools
import os
from tqdm import tqdm
import sqlite3
import datetime
import pickle
import inspect
import numpy as np
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 


import Artha
from Artha.twitter import TwitterAPI
from Artha.sqlitedb import TSQLite
import Artha.configs.twitter_config as c
from Artha.extraction import *

checkra = TwitterAPI(username = "checkra_", bearer_token = c.c_bearer, key = c.c_key, secret = c.c_secret, token = c.c_token, token_secret = c.c_token_secret)
print(checkra.id,"done")

In [ ]:
checkra.user_lookup("MetreSteven")

In [ ]:
# tweets = checkra.get_recent_tweets("MetreSteven")
# print(len(tweets))
# tweet_text = [t["full_text"] for t in tweets]
# print(tweet_text[0])

In [ ]:
checkra_follows = checkra.get_following1("checkra_")


In [ ]:
checkra.user_lookup(checkra_follows[68])["username"]

In [ ]:
for i in tqdm(checkra_follows[264+62:]):
    cur_user = checkra.user_lookup(i)["username"]
    cur_tweets = checkra.get_recent_tweets(cur_user)
    if cur_tweets != -1:
        with open("../data/tweets/u"+cur_user+"tweets.json", "w+") as w:
            json.dump(cur_tweets, w, indent = 1)
            print(cur_user, "got tweets")

In [ ]:
l = os.listdir("../data/tweets")
for ind, i in enumerate(l):
    if "gooner" in i:
        print(ind)

In [ ]:
with open("../data/tweets/unebraskangooner.json", "r") as r:
    tweets = json.load(r)

In [ ]:
tweets[0]

In [ ]:
# TODO Download recent tweets for all people I follow

In [ ]:
with open("kaleotweets.json", "w+") as w:
    json.dump(tweets, w, indent = 1)

In [ ]:
# goes through entire set of recent tweets, extracts tickers and saves to file
# for debugging ticker extraction
final_ticks, final_names, final_cash = [], [], []
with open("has.txt", "w+") as w, open("not.txt", "w+") as x:
    for ind, i in enumerate(tweet_text):
        tick = extract_crypto_tickers(i)
        name = extract_crypto_names(i)
        cash = extract_cashtags(i)

        final_ticks.extend(tick)
        final_names.extend(name)
        final_cash.extend([c[1:] for c in cash])

        if tick or name:
            w.write(i+"\n")
            json.dump(tick, w)
            w.write("\n")
            json.dump(name, w)
            w.write("\n\n\n")
        else:
            x.write(str(ind)+".  "+i+"\n\n")